# PART 3: Model Development

## C Model

In [1]:
"""
"""

import os
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)

working_directory = 'D:/machine_learning/DFS/NHL/'
os.chdir(working_directory)
data_dir = 'Data/' #Where is your data located?
etl_dir = 'Data/ETL/' #Where is your output data going?

player_stats = pd.read_csv('Data/alldata_2016-2022.csv', index_col = 0) #Read In Our Main Dataset
c_df = pd.read_csv(etl_dir + 'c_stats.csv', index_col = 0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
c_df['HomeOrAway'] = encoder.fit_transform(c_df['HomeOrAway'].to_numpy().reshape(-1, 1))

c_df = c_df.rename(columns={'Team_x' : 'Team'})

# convert date from object dtype to datetime
c_df['Date'] = pd.to_datetime(c_df['Date'], format = '%Y%m%d')
c_df['Date'] = c_df['Date'].dt.strftime('%Y%m%d')

#C DK PTS Rank For The Given Season & Date Pair
c_df['Act_C_FPRank'] = c_df.groupby(['Season','Date'])['FantasyPointsFanDuel'].rank(method='min', ascending = False)

#Columns We Want To Add To Dataset
keep_cols = ['Season','Date','Name','Act_C_FPRank']

#Make sure we have no duplicated columns or infinity errors
c_df = c_df.loc[:,~c_df.columns.duplicated()]
c_df= c_df.replace([np.inf, -np.inf], np.nan)

#Columns We Can't Include In Our Features Datasets
dcols = ['TeamID', 'PlayerID', 'Team', 'Position', 'Games', 'Started', 
         'Goals', 'Assists', 'Points', 'PlusMinus', 'HatTricks',
       'PenaltyMinutes', 'PowerPlayGoals', 'ShortHandedGoals', 'ShotsOnGoal',
       'Blocks',
       'Month', 'Year'
]


# g_vs_act.drop_duplicates(subset=['Player', 'Date'], keep='first', inplace = True, ignore_index = True)

X = c_df.drop(dcols, axis = 1)
Y = c_df['FantasyPointsFanDuel']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

more_dcols = ['Season', 'Date', 'Name', 'Opponent', 'FantasyPointsFanDuel', 'Act_C_FPRank']

X_train.drop(more_dcols, axis = 1, inplace = True)
X_test.drop(more_dcols, axis = 1, inplace = True)

# dump non-scaled train df for external scaling to work
filename = 'scalers/c_X_train_pt.pkl'
joblib.dump(X_train, filename)

# Scaling
from sklearn.preprocessing import StandardScaler
c_scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(c_scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(c_scaler.transform(X_test), columns = X_test.columns)
filename = 'scalers/c_scaler_pt.pkl'
joblib.dump(c_scaler, filename)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

C:\Users\Greg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Training set size: 59307
Testing set size: 19769

Num Possible Features: 90


C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [2]:
# hyperparameter tuning function
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

def hyperparameter_tuning(feat_set):
    # define model for param tuning
    model = LGBMRegressor()

    # define param grid
    param_grid = {
        'random_state' : [1],
        'n_jobs' : [-1],
        'n_estimators' : [1000],
        'learning_rate': [0.001, 0.01, 0.1],
        'num_leaves': [15, 31, 62, 93, 127],
        'max_depth' :[-1, 20, 40],
        'min_child_samples' : [20, 80],
        'max_bin' : [63],
        'device' : ['gpu']
    }

    gsearch = GridSearchCV(model, param_grid, verbose = 10, scoring = 'neg_mean_absolute_error', cv = 5) 

    lgb_model = gsearch.fit(X_train[feat_set], Y_train)
    print('best params')
    print(gsearch.best_params_, gsearch.best_score_)
    preds_lgb_model = lgb_model.predict(X_test[feat_set])
    mae_lgb = mean_absolute_error(Y_test, preds_lgb_model)
    print(" MAE: %f" % (mae_lgb ))
    return lgb_model.best_params_

In [3]:
# hyperparameter tuning for all features
best_params_all = hyperparameter_tuning(X_train.columns.tolist())

""" MODEL SELECTION (ALL FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_all)
"""                 """

#print possible features
print('possible features:', X_train.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Season','Date','Opponent','Name','FantasyPointsFanDuel']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/LGBM_models/C_model_allfeats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.128 total time=   4.5s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.187 total time=   3.8s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.119 total time=  40.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.131 total time=  40.9s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.129 total time=   5.5s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.165 total time=  37.3s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.075 total time=  38.0s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.112 total time=  38.8s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.132 total time=  30.1s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.114 total time=  40.8s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.170 total time=  41.0s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.076 total time=  18.2s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.115 total time=  19.5s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.127 total time=  20.0s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.084 total time=  23.2s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.027 total time=  23.8s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.016 total time=  24.1s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.129 total time=  20.7s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.145 total time=  20.8s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.089 total time=  21.7s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.075 total time=  17.5s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.092 total time=  24.3s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.098 total time=  24.1s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.064 total time=  16.7s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.141 total time=  16.6s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.133 total time=  23.2s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/C_model_allfeats_pt.pkl']

In [4]:
# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# hyperparameter tuning for 50 features
best_params_50 = hyperparameter_tuning(attr50)

""" MODEL SELECTION (50 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_50)
"""                 """

#print top50 features
print('T50 features', attr50, '\n')

# Fit model, make predictions with t50 features
model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])

# add model performance to pdf dataframe
pdf['Pred_FP_50'] = preds50

filename = 'models/LGBM_models/C_model_50feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.128 total time=   3.7s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.187 total time=   3.6s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.117 total time=  28.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.132 total time=  25.8s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.129 total time=   3.6s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.166 total time=  29.1s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.075 total time=  24.2s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.112 total time=  25.5s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.132 total time=  19.7s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.112 total time=  27.6s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.169 total time=  25.3s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.077 total time=  20.9s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.115 total time=  18.9s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.125 total time=  25.1s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.089 total time=  22.6s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.028 total time=  21.2s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.029 total time=  21.8s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.135 total time=  20.7s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.154 total time=  20.7s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.094 total time=  21.1s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.106 total time=  15.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.086 total time=  21.3s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.097 total time=  21.4s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.052 total time=  15.4s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.143 total time=  15.4s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.142 total time=  21.0s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/C_model_50feats_pt.pkl']

In [5]:
# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# hyperparameter tuning for 30 features
best_params_30 = hyperparameter_tuning(attr30)

""" MODEL SELECTION (30 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_30)
"""                 """

#print top30 features
print('T30 features', attr30, '\n')

# Fit model, make predictions with t30 features
model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])

# add model performance to pdf dataframe
pdf['Pred_FP_30'] = preds30

# dump model
filename = 'models/LGBM_models/C_model_30feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.128 total time=   3.3s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.188 total time=   3.4s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.117 total time=  22.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.128 total time=  23.8s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.129 total time=   3.6s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.165 total time=  23.1s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.072 total time=  24.2s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.113 total time=  21.9s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.128 total time=  17.4s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.110 total time=  23.0s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.168 total time=  24.0s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.073 total time=  16.9s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.115 total time=  16.9s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.126 total time=  16.6s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.116 total time=  20.9s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.031 total time=  22.0s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.063 total time=  21.7s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.173 total time=  20.2s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.208 total time=  20.9s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.070 total time=  22.0s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.077 total time=  16.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.087 total time=  21.9s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.096 total time=  21.2s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.069 total time=  14.8s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.130 total time=  14.8s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.162 total time=  22.3s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/C_model_30feats_pt.pkl']

In [6]:
# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

# hyperparameter tuning for 20 features
best_params_20 = hyperparameter_tuning(attr20)

""" MODEL SELECTION (20 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_20)
"""                 """

#print top20 features
print('T20 features', attr20, '\n')

# Fit model, make predictions with t20 features
model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])

# add model performance to pdf dataframe
pdf['Pred_FP_20'] = preds20

# dump model
filename = 'models/LGBM_models/C_model_20feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.128 total time=   3.2s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.189 total time=   3.2s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.118 total time=  22.0s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.123 total time=  22.2s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.128 total time=   3.2s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.167 total time=  21.5s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.072 total time=  21.6s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.109 total time=  21.6s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.125 total time=  16.7s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.111 total time=  22.1s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.170 total time=  22.2s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.074 total time=  16.5s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.112 total time=  16.2s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.122 total time=  16.3s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.125 total time=  20.1s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.039 total time=  20.7s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.046 total time=  20.0s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.180 total time=  19.5s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.172 total time=  19.6s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.087 total time=  19.6s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.082 total time=  14.8s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.087 total time=  20.0s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.103 total time=  20.2s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.091 total time=  14.6s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.123 total time=  14.7s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.146 total time=  19.7s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/C_model_20feats_pt.pkl']

In [7]:
# Using Top 20 Features, Find Top 10 Features
model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr10 = dset['attr'][0:10].tolist()

# hyperparameter tuning for 10 features
best_params_10 = hyperparameter_tuning(attr10)

""" MODEL SELECTION (10 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_10)
"""                 """

#print top10 features
print('T10 features', attr10, '\n')

# Fit model, make predictions with t10 features
model.fit(X_train[attr10], Y_train)
preds10 = model.predict(X_test[attr10])

# add model performance to pdf dataframe
pdf['Pred_FP_10'] = preds10

# dump model
filename = 'models/LGBM_models/C_model_10feats_pt.pkl'
joblib.dump(model, filename)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.140 total time=   2.9s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.194 total time=   2.9s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.125 total time=  21.3s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.150 total time=  21.2s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.140 total time=   2.9s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.171 total time=  20.9s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.092 total time=  20.8s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.121 total time=  20.9s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.150 total time=  15.9s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.124 total time=  21.4s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.177 total time=  21.5s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.092 total time=  15.8s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.122 total time=  15.6s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.146 total time=  15.7s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.196 total time=  19.3s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.134 total time=  19.1s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.124 total time=  19.2s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.296 total time=  19.0s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.297 total time=  19.0s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.220 total time=  19.0s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.141 total time=  14.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.222 total time=  19.2s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.191 total time=  19.2s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.147 total time=  14.1s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.232 total time=  14.2s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.312 total time=  19.2s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/C_model_10feats_pt.pkl']

In [8]:
pdf.to_csv(etl_dir + 'c_predictions_lgbm_50_30_20_10.csv')

feature_sets = ['all', '50', '30', '20', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
display(results_df)

,Features,MAE
0,all,5.85
1,50,5.85
2,30,5.85
3,20,5.86
4,10,5.88


In [9]:
pdf[['Name', 'FantasyPointsFanDuel', 'Pred_FP_all', 'Pred_FP_50', 'Pred_FP_30', 'Pred_FP_20', 'Pred_FP_10']]

,Name,FantasyPointsFanDuel,FantasyPointsFanDuel,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_10
0,J.T. Miller,1.6,1.6,7.877096,7.957331,7.789436,8.070024,6.846416
1,Reid Boucher,0.0,0.0,3.450165,3.472656,3.471178,3.508932,3.937013
2,Noah Gregor,13.6,13.6,3.893036,3.906892,3.925896,3.711822,3.716356
3,Sidney Crosby,3.2,3.2,20.238475,20.417322,20.444856,20.027839,19.972748
4,Patrick Marleau,3.2,3.2,7.729085,7.652211,7.701094,7.962642,7.266119
...,...,...,...,...,...,...,...,...
19764,Eric Staal,23.7,23.7,8.783122,8.825076,9.008491,8.799059,9.107194
19765,Brock Nelson,1.6,1.6,7.312134,7.331875,7.376658,7.474866,7.393188
19766,Travis Boyd,9.6,9.6,4.598667,4.682475,4.420834,4.379369,4.101616
19767,Tyler Seguin,6.4,6.4,13.381809,13.150271,12.634076,12.540439,14.987734


## W Model

In [10]:
"""
"""

import os
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)

working_directory = 'D:/machine_learning/DFS/NHL/'
os.chdir(working_directory)
data_dir = 'Data/' #Where is your data located?
etl_dir = 'Data/ETL/' #Where is your output data going?

player_stats = pd.read_csv('Data/alldata_2016-2022.csv', index_col = 0) #Read In Our Main Dataset
w_df = pd.read_csv(etl_dir + 'w_stats.csv', index_col = 0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
w_df['HomeOrAway'] = encoder.fit_transform(w_df['HomeOrAway'].to_numpy().reshape(-1, 1))

w_df = w_df.rename(columns={'Team_x' : 'Team'})

# convert date from object dtype to datetime
w_df['Date'] = pd.to_datetime(w_df['Date'], format = '%Y%m%d')
w_df['Date'] = w_df['Date'].dt.strftime('%Y%m%d')

#W DK PTS Rank For The Given Season & Date Pair
w_df['Act_W_FPRank'] = w_df.groupby(['Season','Date'])['FantasyPointsFanDuel'].rank(method='min', ascending = False)

#Columns We Want To Add To Dataset
keep_cols = ['Season','Date','Name','Act_W_FPRank']

#Make sure we have no duplicated columns or infinity errors
w_df = w_df.loc[:,~w_df.columns.duplicated()]
w_df= w_df.replace([np.inf, -np.inf], np.nan)

#Columns We Can't Include In Our Features Datasets
dcols = ['TeamID', 'PlayerID', 'Team', 'Position', 'Games', 'Started', 
         'Goals', 'Assists', 'Points', 'PlusMinus', 'HatTricks',
       'PenaltyMinutes', 'PowerPlayGoals', 'ShortHandedGoals', 'ShotsOnGoal',
       'Blocks',
       'Month', 'Year'
]


# g_vs_act.drop_duplicates(subset=['Player', 'Date'], keep='first', inplace = True, ignore_index = True)

X = w_df.drop(dcols, axis = 1)
Y = w_df['FantasyPointsFanDuel']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

more_dcols = ['Season', 'Date', 'Name', 'Opponent', 'FantasyPointsFanDuel', 'Act_W_FPRank']

X_train.drop(more_dcols, axis = 1, inplace = True)
X_test.drop(more_dcols, axis = 1, inplace = True)

# dump non-scaled train df for external scaling to work
filename = 'scalers/w_X_train_pt.pkl'
joblib.dump(X_train, filename)

# Scaling
from sklearn.preprocessing import StandardScaler
w_scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(w_scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(w_scaler.transform(X_test), columns = X_test.columns)
filename = 'scalers/w_scaler_pt.pkl'
joblib.dump(w_scaler, filename)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

C:\Users\Greg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Training set size: 60550
Testing set size: 20184

Num Possible Features: 90


C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
# hyperparameter tuning function
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

def hyperparameter_tuning(feat_set):
    # define model for param tuning
    model = LGBMRegressor()

    # define param grid
    param_grid = {
        'random_state' : [1],
        'n_jobs' : [-1],
        'n_estimators' : [1000],
        'learning_rate': [0.001, 0.01, 0.1],
        'num_leaves': [15, 31, 62, 93, 127],
        'max_depth' :[-1, 20, 40],
        'min_child_samples' : [20, 80],
        'max_bin' : [63],
        'device' : ['gpu']
    }

    gsearch = GridSearchCV(model, param_grid, verbose = 10, scoring = 'neg_mean_absolute_error', cv = 5) 

    lgb_model = gsearch.fit(X_train[feat_set], Y_train)
    print('best params')
    print(gsearch.best_params_, gsearch.best_score_)
    preds_lgb_model = lgb_model.predict(X_test[feat_set])
    mae_lgb = mean_absolute_error(Y_test, preds_lgb_model)
    print(" MAE: %f" % (mae_lgb ))
    return lgb_model.best_params_

In [12]:
# hyperparameter tuning for all features
best_params_all = hyperparameter_tuning(X_train.columns.tolist())

""" MODEL SELECTION (ALL FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_all)
"""                 """

#print possible features
print('possible features:', X_train.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Season','Date','Opponent','Name','FantasyPointsFanDuel']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/LGBM_models/W_model_allfeats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-5.999 total time=   3.6s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.067 total time=   3.7s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.103 total time=  24.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.073 total time=  24.3s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-5.999 total time=   3.7s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.047 total time=  23.6s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.069 total time=  23.8s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.101 total time=  23.7s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.073 total time=  18.6s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.986 total time=  24.0s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.046 total time=  23.8s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.070 total time=  17.9s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.103 total time=  17.9s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.068 total time=  18.0s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.996 total time=  21.5s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.017 total time=  21.5s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.022 total time=  21.8s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.023 total time=  20.6s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.057 total time=  20.8s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.069 total time=  20.8s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-5.985 total time=  16.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.955 total time=  21.7s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.000 total time=  21.6s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.036 total time=  15.9s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.008 total time=  15.9s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.005 total time=  21.1s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/W_model_allfeats_pt.pkl']

In [13]:
# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# hyperparameter tuning for 50 features
best_params_50 = hyperparameter_tuning(attr50)

""" MODEL SELECTION (50 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_50)
"""                 """

#print top50 features
print('T50 features', attr50, '\n')

# Fit model, make predictions with t50 features
model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])

# add model performance to pdf dataframe
pdf['Pred_FP_50'] = preds50

filename = 'models/LGBM_models/W_model_50feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-5.998 total time=   3.4s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.067 total time=   3.4s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.105 total time=  23.6s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.074 total time=  23.6s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-5.998 total time=   3.4s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.043 total time=  22.9s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.068 total time=  22.7s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.100 total time=  22.7s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.073 total time=  17.5s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.987 total time=  23.3s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.046 total time=  23.3s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.069 total time=  17.3s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.103 total time=  17.3s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.068 total time=  17.1s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.004 total time=  21.1s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.055 total time=  21.0s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.023 total time=  21.1s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.010 total time=  20.2s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.053 total time=  20.3s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.098 total time=  20.3s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-5.992 total time=  15.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.953 total time=  21.2s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.983 total time=  21.2s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.075 total time=  15.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.016 total time=  15.5s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.008 total time=  20.6s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/W_model_50feats_pt.pkl']

In [14]:
# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# hyperparameter tuning for 30 features
best_params_30 = hyperparameter_tuning(attr30)

""" MODEL SELECTION (30 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_30)
"""                 """

#print top30 features
print('T30 features', attr30, '\n')

# Fit model, make predictions with t30 features
model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])

# add model performance to pdf dataframe
pdf['Pred_FP_30'] = preds30

# dump model
filename = 'models/LGBM_models/W_model_30feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.003 total time=   3.3s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.075 total time=   3.3s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.112 total time=  23.3s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.076 total time=  23.4s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.003 total time=   3.2s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.046 total time=  22.4s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.083 total time=  22.5s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.115 total time=  22.5s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.077 total time=  17.4s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.985 total time=  23.4s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.056 total time=  23.3s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.083 total time=  16.9s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.116 total time=  16.9s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.072 total time=  16.9s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.037 total time=  20.3s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.057 total time=  20.4s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.091 total time=  20.3s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.036 total time=  19.6s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.083 total time=  19.8s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.138 total time=  19.7s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.000 total time=  15.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.943 total time=  20.8s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.019 total time=  20.7s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.081 total time=  14.9s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.032 total time=  14.8s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.038 total time=  20.1s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/W_model_30feats_pt.pkl']

In [15]:
# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

# hyperparameter tuning for 20 features
best_params_20 = hyperparameter_tuning(attr20)

""" MODEL SELECTION (20 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_20)
"""                 """

#print top20 features
print('T20 features', attr20, '\n')

# Fit model, make predictions with t20 features
model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])

# add model performance to pdf dataframe
pdf['Pred_FP_20'] = preds20

# dump model
filename = 'models/LGBM_models/W_model_20feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.002 total time=   3.0s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.076 total time=   3.0s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.118 total time=  22.0s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.070 total time=  22.1s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.002 total time=   3.0s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.052 total time=  21.3s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.079 total time=  21.5s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.111 total time=  21.5s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.074 total time=  16.3s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.986 total time=  22.1s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.057 total time=  22.0s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.080 total time=  16.0s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.117 total time=  16.0s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.070 total time=  16.0s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.025 total time=  19.6s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.056 total time=  19.9s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.104 total time=  19.9s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.068 total time=  19.2s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.108 total time=  19.0s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.121 total time=  19.1s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.009 total time=  14.5s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.996 total time=  19.9s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.033 total time=  19.6s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.092 total time=  14.4s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.068 total time=  14.4s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.074 total time=  19.5s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/W_model_20feats_pt.pkl']

In [16]:
# Using Top 20 Features, Find Top 10 Features
model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr10 = dset['attr'][0:10].tolist()

# hyperparameter tuning for 10 features
best_params_10 = hyperparameter_tuning(attr10)

""" MODEL SELECTION (10 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_10)
"""                 """

#print top10 features
print('T10 features', attr10, '\n')

# Fit model, make predictions with t10 features
model.fit(X_train[attr10], Y_train)
preds10 = model.predict(X_test[attr10])

# add model performance to pdf dataframe
pdf['Pred_FP_10'] = preds10

# dump model
filename = 'models/LGBM_models/W_model_10feats_pt.pkl'
joblib.dump(model, filename)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.001 total time=   3.0s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.076 total time=   2.9s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.122 total time=  21.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.077 total time=  21.8s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-6.001 total time=   3.0s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.052 total time=  21.1s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.079 total time=  21.1s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.113 total time=  21.0s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.078 total time=  16.2s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.982 total time=  21.7s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.056 total time=  21.7s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.081 total time=  16.0s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.118 total time=  16.0s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.074 total time=  15.9s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.116 total time=  19.2s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.111 total time=  19.1s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.133 total time=  19.1s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.048 total time=  19.0s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.172 total time=  19.6s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.207 total time=  19.8s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.010 total time=  14.1s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-5.969 total time=  19.2s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.111 total time=  19.3s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.138 total time=  14.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-6.049 total time=  14.1s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-6.054 total time=  19.2s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/W_model_10feats_pt.pkl']

In [17]:
pdf.to_csv(etl_dir + 'w_predictions_lgbm_50_30_20_10.csv')

feature_sets = ['all', '50', '30', '20', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
display(results_df)

,Features,MAE
0,all,5.81
1,50,5.81
2,30,5.82
3,20,5.82
4,10,5.81


In [18]:
pdf[['Name', 'FantasyPointsFanDuel', 'Pred_FP_all', 'Pred_FP_50', 'Pred_FP_30', 'Pred_FP_20', 'Pred_FP_10']]

,Name,FantasyPointsFanDuel,FantasyPointsFanDuel,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_10
0,Timo Meier,3.2,3.2,4.538982,4.524791,4.420499,4.404744,4.360592
1,Martin Frk,4.8,4.8,3.958020,3.825442,3.587700,3.443507,3.349157
2,Kyle Palmieri,0.0,0.0,9.781977,8.966216,11.178226,11.010419,10.735324
3,Thomas Vanek,8.5,8.5,6.012748,6.151801,6.057019,6.055845,6.191852
4,Kris Versteeg,22.1,22.1,6.693900,7.037624,6.811218,6.793311,6.496670
...,...,...,...,...,...,...,...,...
20179,Taylor Leier,1.6,1.6,5.485797,5.357495,6.425333,6.197333,6.083351
20180,Joonas Donskoi,0.0,0.0,6.848890,7.107526,7.051967,6.823925,6.328158
20181,Patrick Sharp,0.0,0.0,3.227891,3.076979,3.104089,3.588589,2.450990
20182,Josh Bailey,11.7,11.7,9.966743,9.663474,10.269316,9.628224,10.368168


## D Model

In [19]:
"""
"""

import os
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)

working_directory = 'D:/machine_learning/DFS/NHL/'
os.chdir(working_directory)
data_dir = 'Data/' #Where is your data located?
etl_dir = 'Data/ETL/' #Where is your output data going?

player_stats = pd.read_csv('Data/alldata_2016-2022.csv', index_col = 0) #Read In Our Main Dataset
d_df = pd.read_csv(etl_dir + 'd_stats.csv', index_col = 0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
d_df['HomeOrAway'] = encoder.fit_transform(d_df['HomeOrAway'].to_numpy().reshape(-1, 1))

d_df = d_df.rename(columns={'Team_x' : 'Team'})

# convert date from object dtype to datetime
d_df['Date'] = pd.to_datetime(d_df['Date'], format = '%Y%m%d')
d_df['Date'] = d_df['Date'].dt.strftime('%Y%m%d')

#D DK PTS Rank For The Given Season & Date Pair
d_df['Act_D_FPRank'] = d_df.groupby(['Season','Date'])['FantasyPointsFanDuel'].rank(method='min', ascending = False)

#Columns We Want To Add To Dataset
keep_cols = ['Season','Date','Name','Act_D_FPRank']

#Make sure we have no duplicated columns or infinity errors
d_df = d_df.loc[:,~d_df.columns.duplicated()]
d_df= d_df.replace([np.inf, -np.inf], np.nan)

#Columns We Can't Include In Our Features Datasets
dcols = ['TeamID', 'PlayerID', 'Team', 'Position', 'Games', 'Started', 
         'Goals', 'Assists', 'Points', 'PlusMinus', 'HatTricks',
       'PenaltyMinutes', 'PowerPlayGoals', 'ShortHandedGoals', 'ShotsOnGoal',
       'Blocks',
       'Month', 'Year'
]


# g_vs_act.drop_duplicates(subset=['Player', 'Date'], keep='first', inplace = True, ignore_index = True)

X = d_df.drop(dcols, axis = 1)
Y = d_df['FantasyPointsFanDuel']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

more_dcols = ['Season', 'Date', 'Name', 'Opponent', 'FantasyPointsFanDuel', 'Act_D_FPRank']

X_train.drop(more_dcols, axis = 1, inplace = True)
X_test.drop(more_dcols, axis = 1, inplace = True)

# dump non-scaled train df for external scaling to work
filename = 'scalers/d_X_train_pt.pkl'
joblib.dump(X_train, filename)

# Scaling
from sklearn.preprocessing import StandardScaler
d_scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(d_scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(d_scaler.transform(X_test), columns = X_test.columns)
filename = 'scalers/d_scaler_pt.pkl'
joblib.dump(d_scaler, filename)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

C:\Users\Greg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Training set size: 65250
Testing set size: 21750

Num Possible Features: 90


C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
# hyperparameter tuning function
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

def hyperparameter_tuning(feat_set):
    # define model for param tuning
    model = LGBMRegressor()

    # define param grid
    param_grid = {
        'random_state' : [1],
        'n_jobs' : [-1],
        'n_estimators' : [1000],
        'learning_rate': [0.001, 0.01, 0.1],
        'num_leaves': [15, 31, 62, 93, 127],
        'max_depth' :[-1, 20, 40],
        'min_child_samples' : [20, 80],
        'max_bin' : [63],
        'device' : ['gpu']
    }

    gsearch = GridSearchCV(model, param_grid, verbose = 10, scoring = 'neg_mean_absolute_error', cv = 5) 

    lgb_model = gsearch.fit(X_train[feat_set], Y_train)
    print('best params')
    print(gsearch.best_params_, gsearch.best_score_)
    preds_lgb_model = lgb_model.predict(X_test[feat_set])
    mae_lgb = mean_absolute_error(Y_test, preds_lgb_model)
    print(" MAE: %f" % (mae_lgb ))
    return lgb_model.best_params_

In [21]:
# hyperparameter tuning for all features
best_params_all = hyperparameter_tuning(X_train.columns.tolist())

""" MODEL SELECTION (ALL FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_all)
"""                 """

#print possible features
print('possible features:', X_train.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Season','Date','Opponent','Name','FantasyPointsFanDuel']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/LGBM_models/D_model_allfeats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.541 total time=   3.6s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.563 total time=   3.6s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.550 total time=  23.4s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.613 total time=  23.4s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.541 total time=   3.6s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.548 total time=  22.6s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.529 total time=  22.7s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.548 total time=  22.8s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.614 total time=  17.5s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.526 total time=  23.1s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.554 total time=  23.3s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.531 total time=  17.1s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.549 total time=  17.4s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.611 total time=  17.1s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.538 total time=  21.0s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.488 total time=  21.0s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.515 total time=  21.1s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.583 total time=  20.4s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.561 total time=  20.4s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.547 total time=  20.3s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.601 total time=  15.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.561 total time=  21.0s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.526 total time=  21.0s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.540 total time=  15.4s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.629 total time=  15.3s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.572 total time=  20.6s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/D_model_allfeats_pt.pkl']

In [22]:
# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# hyperparameter tuning for 50 features
best_params_50 = hyperparameter_tuning(attr50)

""" MODEL SELECTION (50 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_50)
"""                 """

#print top50 features
print('T50 features', attr50, '\n')

# Fit model, make predictions with t50 features
model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])

# add model performance to pdf dataframe
pdf['Pred_FP_50'] = preds50

filename = 'models/LGBM_models/D_model_50feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.542 total time=   3.8s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.563 total time=   3.6s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.552 total time=  23.1s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.613 total time=  23.2s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.543 total time=   3.6s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.548 total time=  22.2s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.534 total time=  22.5s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.549 total time=  22.2s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.614 total time=  17.4s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.530 total time=  22.7s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.555 total time=  23.1s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.536 total time=  16.9s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.551 total time=  16.9s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.613 total time=  16.9s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.535 total time=  20.9s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.509 total time=  20.8s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.535 total time=  20.7s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.593 total time=  20.0s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.582 total time=  20.2s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.564 total time=  20.0s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.596 total time=  15.5s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.551 total time=  20.9s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.541 total time=  20.8s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.545 total time=  15.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.631 total time=  15.1s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.578 total time=  20.4s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/D_model_50feats_pt.pkl']

In [23]:
# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# hyperparameter tuning for 30 features
best_params_30 = hyperparameter_tuning(attr30)

""" MODEL SELECTION (30 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_30)
"""                 """

#print top30 features
print('T30 features', attr30, '\n')

# Fit model, make predictions with t30 features
model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])

# add model performance to pdf dataframe
pdf['Pred_FP_30'] = preds30

# dump model
filename = 'models/LGBM_models/D_model_30feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.544 total time=   3.2s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.565 total time=   3.2s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.552 total time=  22.1s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.616 total time=  22.2s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.544 total time=   3.2s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.547 total time=  21.5s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.532 total time=  21.5s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.553 total time=  21.5s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.615 total time=  16.6s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.531 total time=  22.0s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.550 total time=  22.1s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.535 total time=  16.1s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.553 total time=  16.3s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.615 total time=  16.5s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.552 total time=  20.0s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.528 total time=  19.7s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.532 total time=  19.8s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.606 total time=  19.3s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.576 total time=  19.3s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.572 total time=  19.5s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.620 total time=  14.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.536 total time=  20.0s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.551 total time=  19.9s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.563 total time=  14.6s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.639 total time=  14.4s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.597 total time=  19.7s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/D_model_30feats_pt.pkl']

In [24]:
# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

# hyperparameter tuning for 20 features
best_params_20 = hyperparameter_tuning(attr20)

""" MODEL SELECTION (20 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_20)
"""                 """

#print top20 features
print('T20 features', attr20, '\n')

# Fit model, make predictions with t20 features
model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])

# add model performance to pdf dataframe
pdf['Pred_FP_20'] = preds20

# dump model
filename = 'models/LGBM_models/D_model_20feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.544 total time=   3.2s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.564 total time=   3.2s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.552 total time=  22.4s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.618 total time=  22.6s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.544 total time=   3.3s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.546 total time=  21.9s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.534 total time=  21.7s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.553 total time=  21.8s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.617 total time=  16.8s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.531 total time=  22.3s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.551 total time=  22.4s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.535 total time=  16.5s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.553 total time=  16.4s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.615 total time=  16.5s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.556 total time=  20.4s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.555 total time=  20.5s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.562 total time=  20.4s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.601 total time=  19.8s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.607 total time=  19.9s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.608 total time=  20.2s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.630 total time=  15.1s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.551 total time=  20.4s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.560 total time=  20.3s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.579 total time=  14.9s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.657 total time=  14.9s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.607 total time=  20.2s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/D_model_20feats_pt.pkl']

In [25]:
# Using Top 20 Features, Find Top 10 Features
model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr10 = dset['attr'][0:10].tolist()

# hyperparameter tuning for 10 features
best_params_10 = hyperparameter_tuning(attr10)

""" MODEL SELECTION (10 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_10)
"""                 """

#print top10 features
print('T10 features', attr10, '\n')

# Fit model, make predictions with t10 features
model.fit(X_train[attr10], Y_train)
preds10 = model.predict(X_test[attr10])

# add model performance to pdf dataframe
pdf['Pred_FP_10'] = preds10

# dump model
filename = 'models/LGBM_models/D_model_10feats_pt.pkl'
joblib.dump(model, filename)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.550 total time=   3.0s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.575 total time=   3.0s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, 

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.564 total time=  21.5s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.624 total time=  21.5s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-4.551 total time=   3.0s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_s

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.557 total time=  21.4s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.540 total time=  21.0s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.566 total time=  21.3s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.623 total time=  16.0s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.538 total time=  21.6s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.559 total time=  21.6s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.542 total time=  15.8s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.565 total time=  15.8s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.621 total time=  15.8s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_chil

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.616 total time=  20.6s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.612 total time=  21.0s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.606 total time=  20.2s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.664 total time=  21.2s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.637 total time=  20.8s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.658 total time=  19.9s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sampl

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.630 total time=  16.0s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.604 total time=  21.2s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.595 total time=  20.7s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.613 total time=  14.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-4.669 total time=  15.2s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-4.655 total time=  20.9s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=

['models/LGBM_models/D_model_10feats_pt.pkl']

In [26]:
pdf.to_csv(etl_dir + 'd_predictions_lgbm_50_30_20_10.csv')

feature_sets = ['all', '50', '30', '20', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
display(results_df)

,Features,MAE
0,all,4.35
1,50,4.35
2,30,4.35
3,20,4.35
4,10,4.36


In [27]:
pdf[['Name', 'FantasyPointsFanDuel', 'Pred_FP_all', 'Pred_FP_50', 'Pred_FP_30', 'Pred_FP_20', 'Pred_FP_10']]

,Name,FantasyPointsFanDuel,FantasyPointsFanDuel,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_10
0,Zach Werenski,14.4,14.4,2.659563,2.652684,2.558685,2.756007,2.870556
1,Ian Cole,4.8,4.8,6.577091,6.498465,6.770500,6.703540,6.811655
2,Matt Irwin,4.8,4.8,5.701293,5.732229,5.796944,5.738859,5.490988
3,Deryk Engelland,1.6,1.6,5.889258,5.920817,5.974548,6.029370,6.003527
4,MacKenzie Weegar,3.2,3.2,6.825949,6.755379,6.799938,6.684894,6.652827
...,...,...,...,...,...,...,...,...
21745,Brenden Dillon,6.4,6.4,6.485999,5.945584,5.925121,5.960099,6.109762
21746,Michael Stone,9.6,9.6,5.044211,4.882575,4.935059,5.017138,6.275239
21747,Matt Dumba,11.2,11.2,9.308922,9.077943,9.084548,8.909071,9.249010
21748,Kurtis MacDermid,1.6,1.6,4.516414,4.436694,4.382580,4.377890,3.748089


## G Model

In [28]:
"""
"""

import os
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE

# needs hyperparameters
def lgbm_mod(): return LGBMRegressor(random_state = 1, n_estimators = 1000, learning_rate = 0.01, n_jobs = -1)

working_directory = 'D:/machine_learning/DFS/NHL/'
os.chdir(working_directory)
data_dir = 'Data/' #Where is your data located?
etl_dir = 'Data/ETL/' #Where is your output data going?

player_stats = pd.read_csv('Data/alldata_2016-2022.csv', index_col = 0) #Read In Our Main Dataset
g_df = pd.read_csv(etl_dir + 'g_stats.csv', index_col = 0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
g_df['HomeOrAway'] = encoder.fit_transform(g_df['HomeOrAway'].to_numpy().reshape(-1, 1))

g_df = g_df.rename(columns={'Team_x' : 'Team'})

# convert date from object dtype to datetime
g_df['Date'] = pd.to_datetime(g_df['Date'], format = '%Y%m%d')
g_df['Date'] = g_df['Date'].dt.strftime('%Y%m%d')

#G DK PTS Rank For The Given Season & Date Pair
g_df['Act_G_FPRank'] = g_df.groupby(['Season','Date'])['FantasyPointsFanDuel'].rank(method='min', ascending = False)

#Columns We Want To Add To Dataset
keep_cols = ['Season','Date','Name','Act_G_FPRank']

#Make sure we have no duplicated columns or infinity errors
g_df = g_df.loc[:,~g_df.columns.duplicated()]
g_df= g_df.replace([np.inf, -np.inf], np.nan)

#Columns We Can't Include In Our Features Datasets
dcols = [
     'TeamID',
     'PlayerID',
     'Team',
     'Position',
     'Games',
     'Started',
     'GoaltendingWins',
     'GoaltendingLosses',
     'GoaltendingOvertimeLosses',
     'GoaltendingShotsAgainst',
     'GoaltendingGoalsAgainst',
     'GoaltendingSaves',
     'GoaltendingShutouts',
     'GoaltendingGoalsAgainstAverage',
     'GoaltendingSavePercentage',
     'GoaltendingMinutes',
     'Month',
     'Year',
    ]


# g_vs_act.drop_duplicates(subset=['Player', 'Date'], keep='first', inplace = True, ignore_index = True)

X = g_df.drop(dcols, axis = 1)
Y = g_df['FantasyPointsFanDuel']

from sklearn.model_selection import train_test_split

#Create Training and Testing DataSets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42)

X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
Y_train.reset_index(inplace = True, drop=True)
Y_test.reset_index(inplace = True, drop=True)

print('Training set size:', len(X_train))
print('Testing set size:', len(X_test))

pred_df = pd.concat([X_test, Y_test], axis = 1)

more_dcols = ['Season', 'Date', 'Name', 'Opponent', 'FantasyPointsFanDuel', 'Act_G_FPRank']

X_train.drop(more_dcols, axis = 1, inplace = True)
X_test.drop(more_dcols, axis = 1, inplace = True)

# dump non-scaled train df for external scaling to work
filename = 'scalers/g_X_train_pt.pkl'
joblib.dump(X_train, filename)

# Scaling
from sklearn.preprocessing import StandardScaler
g_scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(g_scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(g_scaler.transform(X_test), columns = X_test.columns)
filename = 'scalers/g_scaler_pt.pkl'
joblib.dump(g_scaler, filename)

print('\nNum Possible Features:',len(X_train.columns.tolist()))

C:\Users\Greg\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Training set size: 19128
Testing set size: 6377


C:\Users\Greg\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(



Num Possible Features: 95


In [29]:
# hyperparameter tuning function
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

def hyperparameter_tuning(feat_set):
    # define model for param tuning
    model = LGBMRegressor()

    # define param grid
    param_grid = {
        'random_state' : [1],
        'n_jobs' : [-1],
        'n_estimators' : [1000],
        'learning_rate': [0.001, 0.01, 0.1],
        'num_leaves': [15, 31, 62, 93, 127],
        'max_depth' :[-1, 20, 40],
        'min_child_samples' : [20, 80],
        'max_bin' : [63],
        'device' : ['gpu']
    }

    gsearch = GridSearchCV(model, param_grid, verbose = 10, scoring = 'neg_mean_absolute_error', cv = 5) 

    lgb_model = gsearch.fit(X_train[feat_set], Y_train)
    print('best params')
    print(gsearch.best_params_, gsearch.best_score_)
    preds_lgb_model = lgb_model.predict(X_test[feat_set])
    mae_lgb = mean_absolute_error(Y_test, preds_lgb_model)
    print(" MAE: %f" % (mae_lgb ))
    return lgb_model.best_params_

In [30]:
# hyperparameter tuning for all features
best_params_all = hyperparameter_tuning(X_train.columns.tolist())

""" MODEL SELECTION (ALL FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_all)
"""                 """

#print possible features
print('possible features:', X_train.columns.tolist(), '\n')

# Fit model, make predictions with all features
model.fit(X_train, Y_train)

preds_all = model.predict(X_test)

pdf = pred_df[['Season','Date','Opponent','Name','FantasyPointsFanDuel']].copy()
pdf['Pred_FP_all'] = preds_all

# save the initial model to disk
filename = 'models/LGBM_models/G_model_allfeats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.110 total time=   3.0s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.271 total time=   3.1s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.936 total time=  22.0s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.183 total time=  22.4s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.118 total time=   3.0s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.209 total time=  21.6s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.083 total time=  21.4s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.931 total time=  21.9s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, m

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.187 total time=  17.1s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.080 total time=  23.9s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.229 total time=  22.3s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, m

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.084 total time=  15.5s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.937 total time=  15.5s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.161 total time=  15.3s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_ch

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.044 total time=  20.6s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.928 total time=  20.4s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.827 total time=  20.4s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samp

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.087 total time=  15.9s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.190 total time=  15.9s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.187 total time=  16.0s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.963 total time=  15.1s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.968 total time=  20.5s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.054 total time=  20.5s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sampl

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.916 total time=  14.9s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.101 total time=  14.9s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.127 total time=  20.3s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

['models/LGBM_models/G_model_allfeats_pt.pkl']

In [31]:
# get top 50 features
dset = pd.DataFrame({'attr':X_train.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr50 = dset['attr'][0:50].tolist()

# hyperparameter tuning for 50 features
best_params_50 = hyperparameter_tuning(attr50)

""" MODEL SELECTION (50 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_50)
"""                 """

#print top50 features
print('T50 features', attr50, '\n')

# Fit model, make predictions with t50 features
model.fit(X_train[attr50], Y_train)
preds50 = model.predict(X_test[attr50])

# add model performance to pdf dataframe
pdf['Pred_FP_50'] = preds50

filename = 'models/LGBM_models/G_model_50feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.108 total time=   2.7s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.269 total time=   2.7s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.968 total time=  20.9s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.180 total time=  20.8s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.116 total time=   2.7s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.211 total time=  18.9s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.069 total time=  19.5s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.942 total time=  18.9s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, m

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.173 total time=  15.5s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.073 total time=  20.8s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.203 total time=  20.7s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, m

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.071 total time=  14.3s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.954 total time=  14.6s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.165 total time=  14.4s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_ch

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.016 total time=  19.4s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.000 total time=  20.0s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.846 total time=  19.5s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sam

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.042 total time=  15.8s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.139 total time=  15.4s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.140 total time=  15.6s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.006 total time=  14.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.846 total time=  19.4s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.041 total time=  19.5s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samp

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.947 total time=  14.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.154 total time=  14.1s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.007 total time=  19.4s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sample

['models/LGBM_models/G_model_50feats_pt.pkl']

In [32]:
# Using Top 50 Features, Find Top 30 Features
model.fit(X_train[attr50], Y_train)
dset = pd.DataFrame({'attr':X_train[attr50].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr30 = dset['attr'][0:30].tolist()

# hyperparameter tuning for 30 features
best_params_30 = hyperparameter_tuning(attr30)

""" MODEL SELECTION (30 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_30)
"""                 """

#print top30 features
print('T30 features', attr30, '\n')

# Fit model, make predictions with t30 features
model.fit(X_train[attr30], Y_train)
preds30 = model.predict(X_test[attr30])

# add model performance to pdf dataframe
pdf['Pred_FP_30'] = preds30

# dump model
filename = 'models/LGBM_models/G_model_30feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.153 total time=   2.7s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.299 total time=   2.7s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.998 total time=  21.1s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.206 total time=  22.4s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.160 total time=   2.7s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.263 total time=  19.0s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.136 total time=  18.9s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.006 total time=  18.9s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, 

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.212 total time=  15.5s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.140 total time=  21.4s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.268 total time=  21.0s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, m

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.143 total time=  15.0s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.013 total time=  14.4s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.190 total time=  14.5s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_c

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.156 total time=  20.5s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.133 total time=  19.4s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.888 total time=  19.2s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sam

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.108 total time=  16.6s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.244 total time=  16.8s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.264 total time=  16.5s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.109 total time=  14.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.039 total time=  19.5s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.193 total time=  19.3s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sam

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.081 total time=  14.2s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.274 total time=  14.3s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.168 total time=  19.4s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sampl

['models/LGBM_models/G_model_30feats_pt.pkl']

In [33]:
# Using Top 30 Features, Find Top 20 Features
model.fit(X_train[attr30], Y_train)
dset = pd.DataFrame({'attr':X_train[attr30].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr20 = dset['attr'][0:20].tolist()

# hyperparameter tuning for 20 features
best_params_20 = hyperparameter_tuning(attr20)

""" MODEL SELECTION (20 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_20)
"""                 """

#print top20 features
print('T20 features', attr20, '\n')

# Fit model, make predictions with t20 features
model.fit(X_train[attr20], Y_train)
preds20 = model.predict(X_test[attr20])

# add model performance to pdf dataframe
pdf['Pred_FP_20'] = preds20

# dump model
filename = 'models/LGBM_models/G_model_20feats_pt.pkl'
joblib.dump(model, filename) 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.154 total time=   2.7s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.307 total time=   2.8s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.019 total time=  21.2s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.198 total time=  21.2s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.161 total time=   2.7s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_chil

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.296 total time=  18.9s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.118 total time=  19.6s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.984 total time=  19.5s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, m

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.196 total time=  16.3s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.154 total time=  21.8s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.289 total time=  21.9s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, m

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.117 total time=  14.8s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-9.998 total time=  14.6s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.171 total time=  14.6s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_ch

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.238 total time=  19.2s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.169 total time=  19.2s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-9.934 total time=  19.1s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sam

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.205 total time=  17.2s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.354 total time=  17.1s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.353 total time=  17.0s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.148 total time=  14.2s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.149 total time=  19.2s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.197 total time=  19.4s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sam

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.187 total time=  14.1s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.105 total time=  14.1s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.198 total time=  19.4s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sampl

['models/LGBM_models/G_model_20feats_pt.pkl']

In [34]:
# Using Top 20 Features, Find Top 10 Features
model.fit(X_train[attr20], Y_train)
dset = pd.DataFrame({'attr':X_train[attr20].columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance', ascending=False).reset_index(drop=True)
attr10 = dset['attr'][0:10].tolist()

# hyperparameter tuning for 10 features
best_params_10 = hyperparameter_tuning(attr10)

""" MODEL SELECTION (10 FEATS)"""
# all feature model with tuned hyperparameters
model = LGBMRegressor(**best_params_10)
"""                 """

#print top10 features
print('T10 features', attr10, '\n')

# Fit model, make predictions with t10 features
model.fit(X_train[attr10], Y_train)
preds10 = model.predict(X_test[attr10])

# add model performance to pdf dataframe
pdf['Pred_FP_10'] = preds10

# dump model
filename = 'models/LGBM_models/G_model_10feats_pt.pkl'
joblib.dump(model, filename)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.178 total time=   2.7s
[CV 2/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 2/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.296 total time=   2.6s
[CV 3/5; 1/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 3/5; 1/90] END device=gpu, learning_rate=0.001, max_bin=63

[CV 4/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.036 total time=  21.0s
[CV 5/5; 5/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 5/5; 5/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.205 total time=  20.8s
[CV 1/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1
[CV 1/5; 6/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=15, random_state=1;, score=-10.180 total time=   2.6s
[CV 2/5; 6/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_chil

[CV 2/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.293 total time=  19.2s
[CV 3/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.132 total time=  19.3s
[CV 4/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 10/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.027 total time=  19.1s
[CV 5/5; 10/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=-1, 

[CV 5/5; 14/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.206 total time=  15.4s
[CV 1/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.152 total time=  20.8s
[CV 2/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 15/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.260 total time=  20.9s
[CV 3/5; 15/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, m

[CV 3/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.137 total time=  14.5s
[CV 4/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 4/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.031 total time=  14.5s
[CV 5/5; 19/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 19/90] END device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.214 total time=  14.6s
[CV 1/5; 20/90] START device=gpu, learning_rate=0.001, max_bin=63, max_depth=20, min_c

...

[CV 2/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.463 total time=  18.6s
[CV 3/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.324 total time=  18.7s
[CV 4/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 4/5; 75/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.221 total time=  18.5s
[CV 5/5; 75/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 1/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.363 total time=  17.0s
[CV 2/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.653 total time=  17.0s
[CV 3/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 3/5; 80/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.737 total time=  17.1s
[CV 4/5; 80/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=20, min_child_sa

[CV 5/5; 84/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.334 total time=  13.7s
[CV 1/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.193 total time=  18.6s
[CV 2/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 2/5; 85/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=20, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.381 total time=  18.6s
[CV 3/5; 85/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sam

[CV 4/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.391 total time=  13.8s
[CV 5/5; 89/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1
[CV 5/5; 89/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=93, random_state=1;, score=-10.531 total time=  13.8s
[CV 1/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1
[CV 1/5; 90/90] END device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_samples=80, n_estimators=1000, n_jobs=-1, num_leaves=127, random_state=1;, score=-10.386 total time=  18.9s
[CV 2/5; 90/90] START device=gpu, learning_rate=0.1, max_bin=63, max_depth=40, min_child_sampl

['models/LGBM_models/G_model_10feats_pt.pkl']

In [35]:
pdf.to_csv(etl_dir + 'g_predictions_lgbm_50_30_20_10.csv')

feature_sets = ['all', '50', '30', '20', '10']

mae_values = [
    "{:.2f}".format(mean_absolute_error(Y_test, preds_all)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds50)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds30)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds20)),
    "{:.2f}".format(mean_absolute_error(Y_test, preds10))
]

results_df = pd.DataFrame({'Features' : feature_sets, 'MAE' : mae_values})

results_df.style.hide_index()
display(results_df)

,Features,MAE
0,all,9.63
1,50,9.61
2,30,9.75
3,20,9.79
4,10,9.80


In [36]:
pdf[['Name', 'FantasyPointsFanDuel', 'Pred_FP_all', 'Pred_FP_50', 'Pred_FP_30', 'Pred_FP_20', 'Pred_FP_10']]

,Name,FantasyPointsFanDuel,FantasyPointsFanDuel,Pred_FP_all,Pred_FP_50,Pred_FP_30,Pred_FP_20,Pred_FP_10
0,Frederik Andersen,27.2,27.2,17.700433,18.630381,15.429773,13.718694,15.016783
1,Michal Neuvirth,9.6,9.6,8.478019,7.422881,8.336531,6.800756,5.085946
2,Martin Jones,0.0,0.0,11.616354,9.387062,12.768182,9.891675,15.029107
3,Martin Jones,32.8,32.8,13.960331,13.773232,14.406698,17.233056,17.678259
4,Jimmy Howard,36.0,36.0,13.616750,10.967764,7.228464,12.555500,13.122742
...,...,...,...,...,...,...,...,...
6372,Anton Forsberg,0.0,0.0,2.495440,2.113114,2.104764,2.003458,1.997879
6373,Keith Kinkaid,0.0,0.0,3.623036,4.572579,4.061341,3.634927,4.090978
6374,Antti Raanta,21.6,21.6,7.289804,8.742792,8.572807,7.146593,7.830681
6375,Philipp Grubauer,0.0,0.0,6.326604,8.287101,5.901499,8.482748,4.613464
